first model training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# ✅ Load the dataset
df = pd.read_csv("/content/Final_cricket_data.csv")


# ✅ Convert required columns to numeric (forcing errors='coerce' to handle bad data)
num_columns = [
    "Runs_Scored", "Batting_Strike_Rate", "Balls_Faced", "Matches_Batted",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits"
]

df[num_columns] = df[num_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Fill NaN values with 0 (assuming missing stats mean 0 performance)
df.fillna(0, inplace=True)

# ✅ Fantasy points calculation
df["Fantasy_Points"] = (
    df["Runs_Scored"]  # Batting points
    + np.where(df["Runs_Scored"] >= 100, 16, 0)  # Century bonus
    + np.where((df["Runs_Scored"] >= 50) & (df["Runs_Scored"] < 100), 8, 0)  # Half-century bonus
    + np.where(df["Batting_Strike_Rate"] >= 140, 6,  # Strike rate bonus
               np.where(df["Batting_Strike_Rate"] >= 120, 4,
                        np.where(df["Batting_Strike_Rate"] >= 100, 2, 0)))
    - np.where((df["Runs_Scored"] == 0) & (df["Balls_Faced"] > 0), 2, 0)  # Duck penalty
    + df["Wickets_Taken"] * 25  # Bowling points
    + np.where(df["Four_Wicket_Hauls"] > 0, 8, 0)  # 4-wicket haul
    + np.where(df["Five_Wicket_Hauls"] > 0, 16, 0)  # 5-wicket haul
    + np.where(df["Economy_Rate"] < 5, 6,  # Economy rate bonus
               np.where(df["Economy_Rate"] < 6, 4,
                        np.where(df["Economy_Rate"] < 7, 2,
                                 np.where(df["Economy_Rate"] > 9, -2, 0))))
    + df["Catches_Taken"] * 8  # Fielding points
    + df["Stumpings"] * 12  # Stumping points
)

# ✅ Save updated dataset
df.to_csv("fantasy_ready_dataset.csv", index=False)
print("✅ Fantasy points calculated and saved in 'fantasy_ready_dataset.csv'!")

✅ Fantasy points calculated and saved in 'fantasy_ready_dataset.csv'!


In [ ]:
df

,Player_Name,Credits,Player Type,Team,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,...,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls,Recent_Batting_Avg,Recent_Bowling_Avg,Fantasy_Points
0,Andre Siddharth,5.5,BAT,CHE,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.000,0.000,0.00,0.0,0.0,0.000,0.000,6.0
1,Anshul Kamboj,6.0,BOWL,CHE,3.0,1.0,2.0,0,0,2.0,...,2.0,0,57.000,11.400,30.00,0.0,0.0,0.000,57.000,76.0
2,Deepak Hooda,7.0,BAT,CHE,11.8,1.6,146.5,42.57142857,26.044,113.1,...,1.0,0,31.433,8.854,21.30,0.0,0.0,26.044,31.433,234.7
3,Devon Conway,8.0,WK,CHE,23.0,3.0,924.0,92,48.63,654.0,...,0.0,0,0.000,0.000,0.00,0.0,0.0,0.000,0.000,1016.0
4,Gurjapneet Singh,6.0,BOWL,CHE,5.0,3.0,22.0,0,11,30.0,...,29.0,0,29.100,7.500,23.20,0.0,0.0,11.000,29.100,771.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Sachin Baby,6.5,BAT,SRH,11.0,1.0,119.0,33,23.8,79.0,...,2.0,0,4.000,4.800,5.00,0.0,0.0,23.800,4.000,205.0
228,Simarjeet- Singh,6.0,BOWL,SRH,12.0,3.0,10.0,3,10,12.0,...,11.0,46082,30.270,9.510,19.09,0.0,0.0,0.000,0.000,307.0
229,Travis Head,9.0,BAT,SRH,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.000,0.000,0.00,0.0,0.0,0.000,0.000,6.0
230,Wiaan Mulder,7.0,ALL,SRH,1.0,0.0,9.0,9,9,11.0,...,0.0,0/16,0.000,16.000,0.00,0.0,0.0,0.000,0.000,15.0


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# ✅ Load the dataset
df = pd.read_csv("/content/fantasy_ready_dataset.csv")

# ✅ Convert numerical columns to float
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits"
]

# ✅ Convert columns to numeric (forcing errors='coerce' to handle bad data)
df[num_columns] = df[num_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Fill missing values with 0
df.fillna(0, inplace=True)

# ✅ Select Features and Target
features = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits"
]

target = "Fantasy_Points"  # Ensure you've calculated Fantasy Points beforehand!

X = df[features]
y = df[target]

# ✅ Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Train XGBoost model
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    enable_categorical=False  # Make sure categorical features are not included
)

xgb_model.fit(X_train, y_train)

# ✅ Make predictions
y_pred = xgb_model.predict(X_test)

# ✅ Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"✅ Model Training Completed! MAE: {mae:.2f}")

# ✅ Save the trained model
import pickle
with open("fantasy_model.pkl", "wb") as model_file:
    pickle.dump(xgb_model, model_file)

print("✅ Fantasy Model saved as 'fantasy_model.pkl'")


✅ Model Training Completed! MAE: 84.33
✅ Fantasy Model saved as 'fantasy_model.pkl'


In [ ]:
# prompt: ACCURACY SCORE

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2*100}")


Mean Squared Error: 28478.702013152943
R-squared: 95.43737343943191


In [ ]:
df.columns

Index(['Player_Name', 'Credits', 'Player Type', 'Team', 'Matches_Batted',
       'Not_Outs', 'Runs_Scored', 'Highest_Score', 'Batting_Average',
       'Balls_Faced', 'Batting_Strike_Rate', 'Centuries', 'Half_Centuries',
       'Fours', 'Sixes', 'Catches_Taken', 'Stumpings', 'Matches_Bowled',
       'Balls_Bowled', 'Runs_Conceded', 'Wickets_Taken', 'Best_Bowling_Match',
       'Bowling_Average', 'Economy_Rate', 'Bowling_Strike_Rate',
       'Four_Wicket_Hauls', 'Five_Wicket_Hauls', 'Recent_Batting_Avg',
       'Recent_Bowling_Avg', 'Fantasy_Points'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np

# ✅ Load the dataset
df = pd.read_csv("/content/d1 consistency.csv")


# ✅ Convert required columns to numeric (forcing errors='coerce' to handle bad data)
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
    "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
    "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
    "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
    "Stumpings", "Credits", "Final_Consistency_Score"  # ✅ Added this
]

df[num_columns] = df[num_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Fill NaN values with 0 (assuming missing stats mean 0 performance)
df.fillna(0, inplace=True)

# ✅ Fantasy points calculation
df["Fantasy_Points"] = (
    df["Runs_Scored"]  # Batting points
    + np.where(df["Runs_Scored"] >= 100, 16, 0)  # Century bonus
    + np.where((df["Runs_Scored"] >= 50) & (df["Runs_Scored"] < 100), 8, 0)  # Half-century bonus
    + np.where(df["Batting_Strike_Rate"] >= 140, 6,  # Strike rate bonus
               np.where(df["Batting_Strike_Rate"] >= 120, 4,
                        np.where(df["Batting_Strike_Rate"] >= 100, 2, 0)))
    - np.where((df["Runs_Scored"] == 0) & (df["Balls_Faced"] > 0), 2, 0)  # Duck penalty
    + df["Wickets_Taken"] * 25  # Bowling points
    + np.where(df["Four_Wicket_Hauls"] > 0, 8, 0)  # 4-wicket haul
    + np.where(df["Five_Wicket_Hauls"] > 0, 16, 0)  # 5-wicket haul
    + np.where(df["Economy_Rate"] < 5, 6,  # Economy rate bonus
               np.where(df["Economy_Rate"] < 6, 4,
                        np.where(df["Economy_Rate"] < 7, 2,
                                 np.where(df["Economy_Rate"] > 9, -2, 0))))
    + df["Catches_Taken"] * 8  # Fielding points
    + df["Stumpings"] * 12  # Stumping points
)

# ✅ Save updated dataset
df.to_csv("fantasy_ready_dataset.csv", index=False)
print("✅ Fantasy points calculated and saved in 'fantasy_ready_dataset.csv'!")


In [ ]:
import pandas as pd
import pickle
from google.colab import files  # ✅ For file upload in Google Colab

# ✅ Ask user to upload the playing 22 CSV file
print("🔹 Please upload the 'playing 22' CSV file:")
uploaded = files.upload()

# ✅ Get the uploaded filename
playing_22_csv = list(uploaded.keys())[0]
print(f"✅ File '{playing_22_csv}' uploaded successfully!")

# ✅ Load main player dataset
df_main = pd.read_csv("/content/fantasy_ready_dataset.csv")

# ✅ Load trained fantasy prediction model
model = pickle.load(open("/content/fantasy_model.pkl", "rb"))

# ✅ Function to select best 11 players
def select_best_11(playing_22_csv):
    # ✅ Load playing 22 dataset
    df_playing_22 = pd.read_csv(playing_22_csv)

    # ✅ Keep only "PLAYING" players
    df_playing_22 = df_playing_22[df_playing_22["IsPlaying"] == "PLAYING"]

    print(f"✅ Found {len(df_playing_22)} players marked as 'PLAYING'.")

    # ✅ Merge playing 22 with main dataset
    match_players = df_playing_22.merge(df_main, left_on="Player Name", right_on="Player_Name", how="left")

    # ✅ Fix column renaming issue for "Credits"
    if "Credits_x" in match_players.columns:
        match_players.rename(columns={"Credits_x": "Credits"}, inplace=True)
    elif "Credits_y" in match_players.columns:
        match_players.rename(columns={"Credits_y": "Credits"}, inplace=True)

    # ✅ Drop players not found in main dataset
    match_players = match_players.dropna(subset=["Credits"])

    print(f"✅ Found {len(match_players)} players in the main dataset.")

    # ✅ Debugging print statement
    print("🔹 Columns in match_players:", match_players.columns.tolist())

    # ✅ Ensure all required features exist
    required_features = [
        "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
        "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
        "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
        "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
        "Stumpings", "Credits"
    ]

    # ✅ Check for missing features
    missing_features = [feat for feat in required_features if feat not in match_players.columns]
    if missing_features:
        print(f"❌ Missing features in dataset: {missing_features}")
        return None

    # ✅ Convert numeric columns to float and replace errors
    for col in required_features:
        match_players[col] = pd.to_numeric(match_players[col], errors='coerce').fillna(0)

    # ✅ Predict fantasy points
    X_match = match_players[required_features]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # ✅ Sort players by predicted fantasy points and lineup order
    match_players = match_players.sort_values(by=["Predicted_Fantasy_Points", "lineupOrder"], ascending=[False, True])

    # ✅ Select top 11 players
    best_11 = match_players.head(11)

    # ✅ Assign Captain (C) and Vice-Captain (VC)
    best_11["C/VC"] = "NA"
    best_11.loc[best_11.index[0], "C/VC"] = "C"  # Captain (Highest Points)
    best_11.loc[best_11.index[1], "C/VC"] = "VC"  # Vice-Captain (2nd Highest Points)

    # ✅ Save selected team
    best_11.to_csv("selected_best_11.csv", index=False)
    best_11.to_csv("selected_best_11_with_c_vc.csv", index=False)

    print("✅ Best 11 players selected and saved as 'selected_best_11.csv'!")
    print(f"✅ Captain: {best_11.iloc[0]['Player_Name']} (C)")
    print(f"✅ Vice-Captain: {best_11.iloc[1]['Player_Name']} (VC)")

    return best_11

# ✅ Run team selection
best_11_team = select_best_11(playing_22_csv)

# ✅ Print selected team
if best_11_team is not None:
    print("\n🔹 **Selected Best 11 Players:**")
    print(best_11_team[["Player_Name", "Credits"]])

    # ✅ Provide the file for download
    files.download("selected_best_11.csv")


In [ ]:
import pandas as pd
import numpy as np
import pickle

# ✅ Load consistency dataset (user data)
user_df = pd.read_csv("/content/d1 consistency.csv")

# ✅ Load the trained fantasy model
with open("fantasy_model.pkl", "rb") as f:
    model = pickle.load(f)

# ✅ Ensure required numeric columns
required_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits"
]

# ✅ Ensure numeric type and fill NaNs
user_df[required_columns] = user_df[required_columns].apply(pd.to_numeric, errors="coerce")
user_df.fillna(0, inplace=True)

# ✅ Predict fantasy points using the model
X_user = user_df[required_columns]
user_df['Predicted_Fantasy_Points'] = model.predict(X_user)

# ✅ Select top 11 players based on predicted fantasy points
best_11 = user_df.sort_values(by='Predicted_Fantasy_Points', ascending=False).head(11).copy()

# ✅ Assign Captain and Vice-Captain based on Consistency Score
best_11 = best_11.sort_values(by="Final_Consistency_Score", ascending=False)
best_11.reset_index(drop=True, inplace=True)
best_11.loc[0, "Role"] = "Captain"
best_11.loc[1, "Role"] = "Vice-Captain"
best_11.loc[2:, "Role"] = "Player"

# ✅ Output final team
print("🏏 Final Fantasy 11 Players:")
print(best_11[["Player_Name", "Team", "Player Type", "Predicted_Fantasy_Points", "Final_Consistency_Score", "Role"]])

# ✅ Save final team
best_11.to_csv("final_fantasy_11.csv", index=False)
print("✅ Final team saved to 'final_fantasy_11.csv'")


FileNotFoundError: [Errno 2] No such file or directory: 'fantasy_model.pkl'

In [ ]:
import pandas as pd
import numpy as np

# ✅ Load the dataset
df = pd.read_csv("/content/d1 consistency.csv")


# ✅ Convert required columns to numeric (forcing errors='coerce' to handle bad data)
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
    "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
    "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
    "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
    "Stumpings", "Credits", "Final_Consistency_Score"  # ✅ Added this
]

df[num_columns] = df[num_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Fill NaN values with 0 (assuming missing stats mean 0 performance)
df.fillna(0, inplace=True)

# ✅ Fantasy points calculation
df["Fantasy_Points"] = (
    df["Runs_Scored"]  # Batting points
    + np.where(df["Runs_Scored"] >= 100, 16, 0)  # Century bonus
    + np.where((df["Runs_Scored"] >= 50) & (df["Runs_Scored"] < 100), 8, 0)  # Half-century bonus
    + np.where(df["Batting_Strike_Rate"] >= 140, 6,  # Strike rate bonus
               np.where(df["Batting_Strike_Rate"] >= 120, 4,
                        np.where(df["Batting_Strike_Rate"] >= 100, 2, 0)))
    - np.where((df["Runs_Scored"] == 0) & (df["Balls_Faced"] > 0), 2, 0)  # Duck penalty
    + df["Wickets_Taken"] * 25  # Bowling points
    + np.where(df["Four_Wicket_Hauls"] > 0, 8, 0)  # 4-wicket haul
    + np.where(df["Five_Wicket_Hauls"] > 0, 16, 0)  # 5-wicket haul
    + np.where(df["Economy_Rate"] < 5, 6,  # Economy rate bonus
               np.where(df["Economy_Rate"] < 6, 4,
                        np.where(df["Economy_Rate"] < 7, 2,
                                 np.where(df["Economy_Rate"] > 9, -2, 0))))
    + df["Catches_Taken"] * 8  # Fielding points
    + df["Stumpings"] * 12  # Stumping points
)

# ✅ Save updated dataset
df.to_csv("fantasy_ready_dataset.csv", index=False)
print("✅ Fantasy points calculated and saved in 'fantasy_ready_dataset.csv'!")


✅ Fantasy points calculated and saved in 'fantasy_ready_dataset.csv'!


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# ✅ Load the dataset
df = pd.read_csv("/content/fantasy_ready_dataset.csv")

# ✅ Convert numerical columns to float
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits", "Final_Consistency_Score"  # ✅ Add this line
]

# ✅ Convert columns to numeric (forcing errors='coerce' to handle bad data)
df[num_columns] = df[num_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Fill missing values with 0
df.fillna(0, inplace=True)

# ✅ Select Features and Target

features = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits", "Final_Consistency_Score"  # ✅ Add this too
]


target = "Fantasy_Points"  # Ensure you've calculated Fantasy Points beforehand!

X = df[features]
y = df[target]

# ✅ Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Train XGBoost model
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    enable_categorical=False  # Make sure categorical features are not included
)

xgb_model.fit(X_train, y_train)

# ✅ Make predictions
y_pred = xgb_model.predict(X_test)

# ✅ Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"✅ Model Training Completed! MAE: {mae:.2f}")

# ✅ Save the trained model
import pickle
with open("fantasy_model.pkl", "wb") as model_file:
    pickle.dump(xgb_model, model_file)

print("✅ Fantasy Model saved as 'fantasy_model.pkl'")


✅ Model Training Completed! MAE: 103.86
✅ Fantasy Model saved as 'fantasy_model.pkl'


In [ ]:
import pandas as pd
import pickle
from google.colab import files  # For file upload in Google Colab

# 🔹 Ask user to upload the 'playing 22' CSV file
print("🔹 Please upload the 'playing 22' CSV file:")
uploaded = files.upload()

# ✅ Get the uploaded filename
playing_22_csv = list(uploaded.keys())[0]
print(f"✅ File '{playing_22_csv}' uploaded successfully!")

# ✅ Load main player dataset
df_main = pd.read_csv("/content/fantasy_ready_dataset.csv")

# ✅ Load trained fantasy prediction model
model = pickle.load(open("/content/fantasy_model.pkl", "rb"))

# ✅ Function to select best 11 players
def select_best_11(playing_22_csv):
    # ✅ Load playing 22 dataset
    df_playing_22 = pd.read_csv(playing_22_csv)

    # ✅ Keep only "PLAYING" players
    df_playing_22 = df_playing_22[df_playing_22["IsPlaying"] == "PLAYING"]

    print(f"✅ Found {len(df_playing_22)} players marked as 'PLAYING'.")

    # ✅ Merge playing 22 with main dataset
    match_players = df_playing_22.merge(df_main, left_on="Player Name", right_on="Player_Name", how="left")

    # ✅ Fix column renaming issue for "Credits"
    if "Credits_x" in match_players.columns:
        match_players.rename(columns={"Credits_x": "Credits"}, inplace=True)
    elif "Credits_y" in match_players.columns:
        match_players.rename(columns={"Credits_y": "Credits"}, inplace=True)

    # ✅ Drop players not found in main dataset
    match_players = match_players.dropna(subset=["Credits"])

    print(f"✅ Found {len(match_players)} players in the main dataset.")

    # ✅ Ensure all required features exist
    required_features = [
        "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
        "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
        "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
        "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
        "Stumpings", "Credits"
    ]

    # ✅ Check for missing features
    missing_features = [feat for feat in required_features if feat not in match_players.columns]
    if missing_features:
        print(f"❌ Missing features in dataset: {missing_features}")
        return None

    # ✅ Convert numeric columns to float and handle missing
    for col in required_features:
        match_players[col] = pd.to_numeric(match_players[col], errors='coerce').fillna(0)

    # ✅ Predict fantasy points
    X_match = match_players[required_features]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # ✅ Calculate consistency score
    match_players["Consistency_Score"] = (
        0.4 * match_players["Recent_Batting_Avg"] +
        0.3 * match_players["Recent_Bowling_Avg"] +
        0.1 * match_players["Batting_Average"] +
        0.1 * match_players["Bowling_Average"] +
        0.1 * match_players["Catches_Taken"]
    )

    # ✅ Sort by predicted fantasy points and lineup order
    match_players = match_players.sort_values(by=["Predicted_Fantasy_Points", "lineupOrder"], ascending=[False, True])

    # ✅ Select top 11 players
    best_11 = match_players.head(11).copy()

    # ✅ From top 11, assign Captain and Vice-Captain based on consistency
    best_11 = best_11.sort_values(by="Consistency_Score", ascending=False).reset_index(drop=True)
    best_11["C/VC"] = "NA"
    best_11.loc[0, "C/VC"] = "C"   # Captain
    best_11.loc[1, "C/VC"] = "VC"  # Vice-Captain

    # ✅ Re-sort for final display (by fantasy points)
    best_11 = best_11.sort_values(by="Predicted_Fantasy_Points", ascending=False).reset_index(drop=True)

    # ✅ Save selected team
    best_11.to_csv("selected_best_11.csv", index=False)
    best_11.to_csv("selected_best_11_with_c_vc.csv", index=False)

    print("✅ Best 11 players selected and saved as 'selected_best_11.csv'!")
    print(f"✅ Captain: {best_11[best_11['C/VC'] == 'C']['Player_Name'].values[0]} (C)")
    print(f"✅ Vice-Captain: {best_11[best_11['C/VC'] == 'VC']['Player_Name'].values[0]} (VC)")

    return best_11

# ✅ Run team selection
best_11_team = select_best_11(playing_22_csv)

# ✅ Print selected team
if best_11_team is not None:
    print("\n🔹 Selected Best 11 Players:")
    print(best_11_team[["Player_Name", "Credits", "Predicted_Fantasy_Points", "Consistency_Score", "C/VC"]])

    # ✅ Provide the file for download
    files.download("selected_best_11.csv")


🔹 Please upload the 'playing 22' CSV file:


Saving SquadPlayerNames_IndianT20League - Match_20.csv to SquadPlayerNames_IndianT20League - Match_20.csv
✅ File 'SquadPlayerNames_IndianT20League - Match_20.csv' uploaded successfully!
✅ Found 22 players marked as 'PLAYING'.
✅ Found 25 players in the main dataset.


ValueError: feature_names mismatch: ['Runs_Scored', 'Batting_Average', 'Batting_Strike_Rate', 'Balls_Faced', 'Centuries', 'Half_Centuries', 'Fours', 'Sixes', 'Recent_Batting_Avg', 'Matches_Bowled', 'Wickets_Taken', 'Bowling_Average', 'Economy_Rate', 'Bowling_Strike_Rate', 'Four_Wicket_Hauls', 'Five_Wicket_Hauls', 'Recent_Bowling_Avg', 'Catches_Taken', 'Stumpings', 'Credits', 'Final_Consistency_Score'] ['Runs_Scored', 'Batting_Average', 'Batting_Strike_Rate', 'Balls_Faced', 'Centuries', 'Half_Centuries', 'Fours', 'Sixes', 'Recent_Batting_Avg', 'Matches_Bowled', 'Wickets_Taken', 'Bowling_Average', 'Economy_Rate', 'Bowling_Strike_Rate', 'Four_Wicket_Hauls', 'Five_Wicket_Hauls', 'Recent_Bowling_Avg', 'Catches_Taken', 'Stumpings', 'Credits']
expected Final_Consistency_Score in input data

In [ ]:
import pandas as pd
import pickle
from google.colab import files

# 🔹 Upload CSV
print("\ud83d\udd39 Please upload the 'playing 22' CSV file:")
uploaded = files.upload()
playing_22_csv = list(uploaded.keys())[0]
print(f"\u2705 File '{playing_22_csv}' uploaded successfully!")

# Load datasets
model = pickle.load(open("/content/fantasy_model.pkl", "rb"))
df_main = pd.read_csv("/content/fantasy_ready_dataset.csv")

def select_best_11_team(playing_22_csv):
    df_playing = pd.read_csv(playing_22_csv)
    df_playing = df_playing[df_playing['IsPlaying'] == 'PLAYING']
    print(f"\u2705 Found {len(df_playing)} players marked as 'PLAYING'.")

    match_players = df_playing.merge(df_main, left_on='Player Name', right_on='Player_Name', how='left')
    match_players.dropna(subset=['Credits'], inplace=True)

    required_features = [
        "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
        "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
        "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
        "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
        "Stumpings", "Credits"
    ]

    for col in required_features:
        match_players[col] = pd.to_numeric(match_players[col], errors='coerce').fillna(0)

    match_players["Predicted_Fantasy_Points"] = model.predict(match_players[required_features])
    match_players["Consistency_Score"] = (
        0.4 * match_players["Recent_Batting_Avg"] +
        0.3 * match_players["Recent_Bowling_Avg"] +
        0.1 * match_players["Batting_Average"] +
        0.1 * match_players["Bowling_Average"] +
        0.1 * match_players["Catches_Taken"]
    )

    wk = match_players[match_players['Player_Type'] == 'WK'].sort_values(by="Predicted_Fantasy_Points", ascending=False).head(1)
    bat = match_players[match_players['Player_Type'] == 'BAT'].sort_values(by="Predicted_Fantasy_Points", ascending=False).head(4)
    bowl = match_players[match_players['Player_Type'] == 'BOWL'].sort_values(by="Predicted_Fantasy_Points", ascending=False).head(4)
    allr = match_players[match_players['Player_Type'] == 'ALL'].sort_values(by="Predicted_Fantasy_Points", ascending=False).head(2)

    selected = pd.concat([wk, bat, bowl, allr])

    # Reduce credit if > 100
    while selected['Credits'].sum() > 100:
        over_budget = selected[selected['Credits'] == selected['Credits'].max()]
        to_remove = over_budget.sort_values(by='Consistency_Score').head(1)
        role = to_remove.iloc[0]['Player_Type']

        available = match_players[(match_players['Player_Type'] == role) & (~match_players['Player_Name'].isin(selected['Player_Name']))]
        replacement = available.sort_values(by=['Predicted_Fantasy_Points', 'Consistency_Score'], ascending=[False, False]).head(1)

        if not replacement.empty:
            selected = selected[selected['Player_Name'] != to_remove.iloc[0]['Player_Name']]
            selected = pd.concat([selected, replacement])
        else:
            break

    # Assign C/VC
    selected = selected.sort_values(by=["Consistency_Score", "Predicted_Fantasy_Points"], ascending=False).reset_index(drop=True)
    selected['C/VC'] = 'NA'
    selected.loc[0, 'C/VC'] = 'C'
    selected.loc[1, 'C/VC'] = 'VC'

    # Save and show
    selected.to_csv("selected_best_11.csv", index=False)
    print("\u2705 Best 11 players selected and saved as 'selected_best_11.csv'!")
    print(f"\u2705 Total Credits Used: {selected['Credits'].sum():.2f}")
    print(f"\u2705 Captain: {selected.iloc[0]['Player_Name']}, Vice-Captain: {selected.iloc[1]['Player_Name']}")
    return selected

# Run
team = select_best_11_team(playing_22_csv)
if team is not None:
    print("\n\ud83d\udd39 Final Selected Team:")
    print(team[["Player_Name", "Player_Type", "Credits", "Predicted_Fantasy_Points", "Consistency_Score", "C/VC"]])
    files.download("selected_best_11.csv")

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/zmq/eventloop/zmqstream.py", line 557, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/iostream.py", line 120, in _handle_event
    event_f()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/iostream.py", line 518, in _flush
    self.session.send(
  File "/usr/local/lib/python3.11/dist-packages/jupyter_client/session.py", line 742, in send
    to_send = self.serialize(msg, ident)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/jupyter_client/session.py", line 630, in serialize
    content = self.pack(content)
              ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/jupyter_client/session.py", line 82, in <lambda>
    json_packer = lambda obj: jsonapi.dumps(obj, default=date_default,
                              ^

TypeError: 'NoneType' object is not subscriptable

In [ ]:

import pandas as pd
import pickle
from google.colab import files

# 🔹 Ask user to upload the 'playing 22' CSV file
print("🔹 Please upload the 'playing 22' CSV file:")
uploaded = files.upload()

# ✅ Get the uploaded filename
playing_22_csv = list(uploaded.keys())[0]
print(f"✅ File '{playing_22_csv}' uploaded successfully!")

# ✅ Load main player dataset
df_main = pd.read_csv("/content/fantasy_ready_dataset.csv")
df_main.rename(columns={'Player Type': 'Player_Type'}, inplace=True)

# ✅ Load trained fantasy prediction model
model = pickle.load(open("/content/fantasy_model.pkl", "rb"))

# ✅ Define model input features
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
    "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
    "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
    "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
    "Stumpings", "Credits", "Final_Consistency_Score"
]

# ✅ Role mapping
role_map = {
    "WICKETKEEPER": "WK",
    "BATTER": "BAT",
    "ALLROUNDER": "ALL",
    "BOWLER": "BOWL"
}

# ✅ Define function to select best 11
def select_best_11(playing_22_csv):
    df_playing_22 = pd.read_csv(playing_22_csv)
    df_playing_22 = df_playing_22[df_playing_22["IsPlaying"] == "PLAYING"]
    print(f"✅ Found {len(df_playing_22)} players marked as 'PLAYING'.")

    # 🆕 Use roles and team from uploaded file
    df_playing_22["Player Type"] = df_playing_22["Player_Type"].str.upper().map(role_map)
    df_playing_22.rename(columns={"Player Type": "Player_Type"}, inplace=True)

    # ✅ Merge with main dataset for stats
    match_players = df_playing_22.merge(df_main, left_on="Player Name", right_on="Player_Name", how="left")


    match_players["Player_Type"] = match_players["Player_Type"]

    # ✅ Fix credits
    if "Credits_x" in match_players.columns:
        match_players["Credits"] = match_players["Credits_x"]
    elif "Credits_y" in match_players.columns:
        match_players["Credits"] = match_players["Credits_y"]

    match_players = match_players.dropna(subset=["Credits"])
    print(f"✅ Found {len(match_players)} players in the main dataset.")

    # Ensure all numerical features are present
    missing = [col for col in num_columns if col not in match_players.columns]
    if missing:
        print(f"❌ Missing features: {missing}")
        return None

    # Convert to numeric
    for col in num_columns:
        match_players[col] = pd.to_numeric(match_players[col], errors='coerce').fillna(0)

    # Predict fantasy points
    X_match = match_players[num_columns]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # Sort by predicted points
    match_players = match_players.sort_values(by=["Predicted_Fantasy_Points", "lineupOrder"], ascending=[False, True])

    # Group players by role
    role_groups = {
        "WK": match_players[match_players["Player_Type"] == "WK"],
        "BAT": match_players[match_players["Player_Type"] == "BAT"],
        "ALL": match_players[match_players["Player_Type"] == "ALL"],
        "BOWL": match_players[match_players["Player_Type"] == "BOWL"]
    }

    # Initial selection (max possible from each)
    selected = pd.concat([
        role_groups["WK"].head(2),
        role_groups["BAT"].head(4),
        role_groups["ALL"].head(4),
        role_groups["BOWL"].head(4)
    ])

    # Remove duplicates and re-sort
    selected = selected.drop_duplicates(subset="Player_Name", keep="first")
    selected = selected.sort_values(by="Predicted_Fantasy_Points", ascending=False)

    # Fill remaining slots (up to 11) within 100 credits
    remaining_players = match_players[~match_players["Player_Name"].isin(selected["Player_Name"])]
    for _, row in remaining_players.iterrows():
        temp_team = pd.concat([selected, pd.DataFrame([row])])
        if len(temp_team) > 11:
            continue
        if temp_team["Credits"].sum() <= 100:
            selected = temp_team
        if len(selected) == 11:
            break

    # Final team with 11 players
    if len(selected) != 11:
        print("❌ Unable to form a valid team with 11 players under constraints.")
        return None

    # Ensure at least one WK
    if (selected["Player_Type"] == "WK").sum() == 0:
        print("⚠️ No WICKETKEEPER in selected 11. Forcing inclusion of one...")

        # Get all available WICKETKEEPERS from playing list
        available_wks = match_players[(match_players["Player_Type"] == "WK") &
                                      (~match_players["Player_Name"].isin(selected["Player_Name"]))]

        if not available_wks.empty:
            # Take the best WK based on predicted points
            best_wk = available_wks.sort_values(by="Predicted_Fantasy_Points", ascending=False).iloc[0]

            # Sort current selected players by least predicted points to remove one
            selected = selected.sort_values(by="Predicted_Fantasy_Points", ascending=True).reset_index(drop=True)

            for i in range(len(selected)):
                temp_team = selected.drop(index=i)
                if temp_team["Credits"].sum() + best_wk["Credits"] <= 100:
                    replaced = selected.iloc[i]["Player_Name"]
                    selected = pd.concat([temp_team, pd.DataFrame([best_wk])])
                    selected = selected.sort_values(by="Predicted_Fantasy_Points", ascending=False).reset_index(drop=True)
                    print(f"✅ Added WK '{best_wk['Player_Name']}' by replacing '{replaced}'")
                    break
            else:
                print("❌ Could not adjust team to include a WK under 100 credits.")

    # Assign Captain and Vice-Captain
    selected = selected.sort_values(by=["Final_Consistency_Score", "Predicted_Fantasy_Points"], ascending=[False, False]).reset_index(drop=True)
    selected["C/VC"] = "NA"
    selected.loc[0, "C/VC"] = "C"
    selected.loc[1, "C/VC"] = "VC"

    # ✅ Save to CSV with Player_Type and Team from uploaded CSV
    output_cols = ["Player_Name", "Player_Type", "Team", "Credits", "Predicted_Fantasy_Points", "Final_Consistency_Score", "C/VC"]
    selected[output_cols].to_csv("selected_best_11.csv", index=False)
    print("✅ Final team saved to 'selected_best_11.csv'")

    return selected

# 🔁 Run the function
best_11_team = select_best_11(playing_22_csv)

# ✅ Display selected team and download
if best_11_team is not None:
    print("\n🔹 Selected Best 11 Players:")
    display_cols = ["Player_Name", "Player_Type", "Team", "Credits", "Final_Consistency_Score", "C/VC"]
    print(best_11_team[display_cols].to_string(index=False))
    files.download("selected_best_11.csv")


# ✅ Display selected team and download



In [ ]:
import pandas as pd
import pickle
from google.colab import files

# 🔹 Ask user to upload the 'playing 22' CSV file
print("🔹 Please upload the 'playing 22' CSV file:")
uploaded = files.upload()

# ✅ Get the uploaded filename
playing_22_csv = list(uploaded.keys())[0]
print(f"✅ File '{playing_22_csv}' uploaded successfully!")

# ✅ Load main player dataset
df_main = pd.read_csv("/content/fantasy_ready_dataset.csv")
df_main.rename(columns={'Player Type': 'Player_Type'}, inplace=True)

# ✅ Load trained fantasy prediction model
model = pickle.load(open("/content/fantasy_model.pkl", "rb"))

# ✅ Define model input features
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
    "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
    "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
    "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
    "Stumpings", "Credits", "Final_Consistency_Score"
]

# ✅ Role mapping
role_map = {
    "WICKETKEEPER": "WK",
    "BATTER": "BAT",
    "ALLROUNDER": "ALL",
    "BOWLER": "BOWL"
}

# ✅ Define function to select best 11
def select_best_11(playing_22_csv):
    df_playing_22 = pd.read_csv(playing_22_csv)
    df_playing_22 = df_playing_22[df_playing_22["IsPlaying"] == "PLAYING"]
    print(f"✅ Found {len(df_playing_22)} players marked as 'PLAYING'.")

    # Merge with main dataset
    match_players = df_playing_22.merge(df_main, left_on="Player Name", right_on="Player_Name", how="left")

    # Fix credits
    if "Credits_x" in match_players.columns:
        match_players["Credits"] = match_players["Credits_x"]
    elif "Credits_y" in match_players.columns:
        match_players["Credits"] = match_players["Credits_y"]

    match_players = match_players.dropna(subset=["Credits"])
    print(f"✅ Found {len(match_players)} players in the main dataset.")

    # Standardize roles
    match_players["Player Type"] = match_players["Player Type"].str.upper().map(role_map)
    match_players["Player_Type"] = match_players["Player Type"]

    # Ensure all numerical features are present
    missing = [col for col in num_columns if col not in match_players.columns]
    if missing:
        print(f"❌ Missing features: {missing}")
        return None

    # Convert to numeric
    for col in num_columns:
        match_players[col] = pd.to_numeric(match_players[col], errors='coerce').fillna(0)

    # Predict fantasy points
    X_match = match_players[num_columns]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # Sort by predicted points
    match_players = match_players.sort_values(by=["Predicted_Fantasy_Points", "lineupOrder"], ascending=[False, True])

    # Group players by role
    role_groups = {
        "WK": match_players[match_players["Player_Type"] == "WK"],
        "BAT": match_players[match_players["Player_Type"] == "BAT"],
        "ALL": match_players[match_players["Player_Type"] == "ALL"],
        "BOWL": match_players[match_players["Player_Type"] == "BOWL"]
    }

    # Initial selection (max possible from each)
    selected = pd.concat([
        role_groups["WK"].head(2),
        role_groups["BAT"].head(4),
        role_groups["ALL"].head(4),
        role_groups["BOWL"].head(4)
    ])

    # Remove duplicates and re-sort
    selected = selected.drop_duplicates(subset="Player_Name", keep="first")
    selected = selected.sort_values(by="Predicted_Fantasy_Points", ascending=False)

    # Fill remaining slots (up to 11) within 100 credits
    remaining_players = match_players[~match_players["Player_Name"].isin(selected["Player_Name"])]
    for _, row in remaining_players.iterrows():
        temp_team = pd.concat([selected, pd.DataFrame([row])])
        if len(temp_team) > 11:
            continue
        if temp_team["Credits"].sum() <= 100:
            selected = temp_team
        if len(selected) == 11:
            break

    # Final team with 11 players
    if len(selected) != 11:
        print("❌ Unable to form a valid team with 11 players under constraints.")
        return None

    # Ensure at least one WK
    if (selected["Player_Type"] == "WK").sum() == 0:
        print("⚠️ No WICKETKEEPER in selected 11. Forcing inclusion of one...")

        # Get all available WICKETKEEPERS from playing list
        available_wks = match_players[(match_players["Player_Type"] == "WK") &
                                      (~match_players["Player_Name"].isin(selected["Player_Name"]))]

        if not available_wks.empty:
            # Take the best WK based on predicted points
            best_wk = available_wks.sort_values(by="Predicted_Fantasy_Points", ascending=False).iloc[0]

            # Sort current selected players by least predicted points to remove one
            selected = selected.sort_values(by="Predicted_Fantasy_Points", ascending=True).reset_index(drop=True)

            for i in range(len(selected)):
                temp_team = selected.drop(index=i)
                if temp_team["Credits"].sum() + best_wk["Credits"] <= 100:
                    replaced = selected.iloc[i]["Player_Name"]
                    selected = pd.concat([temp_team, pd.DataFrame([best_wk])])
                    selected = selected.sort_values(by="Predicted_Fantasy_Points", ascending=False).reset_index(drop=True)
                    print(f"✅ Added WK '{best_wk['Player_Name']}' by replacing '{replaced}'")
                    break
            else:
                print("❌ Could not adjust team to include a WK under 100 credits.")

    # Assign Captain and Vice-Captain
    selected = selected.sort_values(by=["Final_Consistency_Score","Predicted_Fantasy_Points"], ascending=[False, False]).reset_index(drop=True)
    selected["C/VC"] = "NA"
    selected.loc[0, "C/VC"] = "C"
    selected.loc[1, "C/VC"] = "VC"

    # ✅ Save to CSV with Player_Type included
    output_cols = ["Player_Name",  "Player_Type", "Credits", "Predicted_Fantasy_Points", "Final_Consistency_Score", "C/VC"]
    selected[output_cols].to_csv("selected_best_11.csv", index=False)
    print("✅ Final team saved to 'selected_best_11.csv'")

# 🔁 Run the function
select_best_11(playing_22_csv)

# ✅ Display selected team and download
if best_11_team is not None:
    print("\n🔹 Selected Best 11 Players:")
    display_cols = ["Player_Name", "Credits", "Final_Consistency_Score", "C/VC"]
    print(best_11_team[display_cols].to_string(index=False))


🔹 Please upload the 'playing 22' CSV file:


Saving SquadPlayerNames_IndianT20League - Match_20.csv to SquadPlayerNames_IndianT20League - Match_20 (17).csv
✅ File 'SquadPlayerNames_IndianT20League - Match_20 (17).csv' uploaded successfully!
✅ Found 22 players marked as 'PLAYING'.
✅ Found 25 players in the main dataset.
⚠️ No WICKETKEEPER in selected 11. Forcing inclusion of one...
✅ Final team saved to 'selected_best_11.csv'

🔹 Selected Best 11 Players:
               Player_Name  Credits  Final_Consistency_Score C/VC
               Rashid-Khan      8.5                 0.000000   NA
               Jos Buttler      9.0               111.582000    C
            Mohammed Shami      8.5                 0.000000   NA
            Mohammed Siraj      8.0                84.916737   NA
               Pat Cummins      8.0                55.323857   NA
          Heinrich Klaasen      9.0                83.785000   NA
             Sai Sudharsan      8.5               108.424000   VC
            K Nitish Reddy      8.0                 0.00000

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#### final

In [ ]:
import pandas as pd

df = pd.read_csv("SquadPlayerNames_IndianT20League - Match_18 (4).csv")
print("📄 Columns in the uploaded CSV:")
print(df.columns.tolist())


In [ ]:
import pandas as pd
import pickle
from google.colab import files

# 🔹 Ask user to upload the 'playing 22' CSV file
print("🔹 Please upload the 'playing 22' CSV file:")
uploaded = files.upload()

# ✅ Get the uploaded filename
playing_22_csv = list(uploaded.keys())[0]
print(f"✅ File '{playing_22_csv}' uploaded successfully!")

# ✅ Load main player dataset
df_main = pd.read_csv("/content/fantasy_ready_dataset.csv")

# ✅ Load trained fantasy prediction model
model = pickle.load(open("/content/fantasy_model.pkl", "rb"))

# ✅ Define model input features
num_columns = [
    "Runs_Scored", "Batting_Average", "Batting_Strike_Rate", "Balls_Faced", "Centuries",
    "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg", "Matches_Bowled",
    "Wickets_Taken", "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate",
    "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg", "Catches_Taken",
    "Stumpings", "Credits", "Final_Consistency_Score"
]

# ✅ Role mapping
role_map = {
    "WICKETKEEPER": "WK",
    "BATTER": "BAT",
    "ALLROUNDER": "ALL",
    "BOWLER": "BOWL"
}

def select_best_11(playing_22_csv):
    df_playing_22 = pd.read_csv(playing_22_csv)
    df_playing_22 = df_playing_22[df_playing_22["IsPlaying"] == "PLAYING"]

    # Merge with main dataset
    match_players = df_playing_22.merge(df_main, left_on="Player Name", right_on="Player_Name", how="left")

    # Use credits from CSV
    if "Credits_x" in match_players.columns:
        match_players["Credits"] = match_players["Credits_x"]
    elif "Credits_y" in match_players.columns:
        match_players["Credits"] = match_players["Credits_y"]

    match_players = match_players.dropna(subset=["Credits"])
    match_players["Player_Type"] = match_players["Player Type_x"].str.upper().map(role_map)

    # Convert features to numeric
    for col in num_columns:
        match_players[col] = pd.to_numeric(match_players[col], errors='coerce').fillna(0)

    # Predict fantasy points
    match_players["Predicted_Fantasy_Points"] = model.predict(match_players[num_columns])
    match_players = match_players.sort_values(by="Predicted_Fantasy_Points", ascending=False)

    # ✅ Step 1: Select 1 best WK
    wk_players = match_players[match_players["Player_Type"] == "WK"]
    if wk_players.empty:
        print("❌ No WICKETKEEPER found in playing 22.")
        return None
    best_wk = wk_players.iloc[0:1]

    # ✅ Step 2: Select best remaining 10 (excluding the WK already selected)
    remaining = match_players[~match_players["Player_Name"].isin(best_wk["Player_Name"])]

    # Sort by predicted points
    remaining = remaining.sort_values(by="Predicted_Fantasy_Points", ascending=False)

    # Greedy selection of players
    selected = [best_wk]
    total_credits = best_wk["Credits"].sum()

    for _, row in remaining.iterrows():
        if len(pd.concat(selected)) == 11:
            break
        if total_credits + row["Credits"] <= 100:
            selected.append(pd.DataFrame([row]))
            total_credits += row["Credits"]

    selected = pd.concat(selected).sort_values(by="Predicted_Fantasy_Points", ascending=False).reset_index(drop=True)

    if len(selected) != 11:
        print("❌ Couldn't form a valid 11-player team under 100 credits.")
        return None

    # Assign C/VC
    selected["C/VC"] = "NA"
    selected.loc[0, "C/VC"] = "C"
    selected.loc[1, "C/VC"] = "VC"

    output_cols = ["Player_Name", "Player_Type", "Credits", "Predicted_Fantasy_Points", "Final_Consistency_Score", "C/VC"]
    selected[output_cols].to_csv("selected_best_11.csv", index=False)
    print("✅ Final team saved to 'selected_best_11.csv'")
    return selected[output_cols]

# 🔁 Run the function
best_11_team = select_best_11(playing_22_csv)

# ✅ Display team and download CSV
if best_11_team is not None:
    print("\n🔹 Selected Best 11 Players:")
    print(best_11_team.to_string(index=False))
    files.download("selected_best_11.csv")



🔹 Please upload the 'playing 22' CSV file:


Saving SquadPlayerNames_IndianT20League - Match_20.csv to SquadPlayerNames_IndianT20League - Match_20 (27).csv
✅ File 'SquadPlayerNames_IndianT20League - Match_20 (27).csv' uploaded successfully!
❌ No WICKETKEEPER found in playing 22.


NameError: name 'wk_players' is not defined

In [5]:
import pandas as pd



# Merge on exact player names
merged_df = pd.read_csv('/content/recent_data.csv')

# Fill NaNs with 0 for consistency calculations
merged_df.fillna(0, inplace=True)

# Ensure numeric columns
for col in ['Avg_bat', 'SR_bat', 'Avg_bowl', 'Econ_bowl', 'SR_bowl']:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)

# Batting Consistency
merged_df['Bat_Consistency'] = (merged_df['Avg_bat'] + merged_df['SR_bat']) / 2

# Bowling Consistency: 150 - avg of [Avg_bowl, Econ, SR_bowl], but only if any of them > 0
def compute_bowl_consistency(row):
    if row['Avg_bowl'] == 0 and row['Econ_bowl'] == 0 and row['SR_bowl'] == 0:
        return 0
    return 150 - ((row['Avg_bowl'] + row['Econ_bowl'] + row['SR_bowl']) / 3)

merged_df['Bowl_Consistency'] = merged_df.apply(compute_bowl_consistency, axis=1)

# Final Consistency Score
def compute_final_consistency(row):
    if row['Bat_Consistency'] > 0 and row['Bowl_Consistency'] > 0:
        return (row['Bat_Consistency'] + row['Bowl_Consistency']) / 2
    elif row['Bat_Consistency'] > 0:
        return row['Bat_Consistency']
    elif row['Bowl_Consistency'] > 0:
        return row['Bowl_Consistency']
    return 0

merged_df['Consistency_Score'] = merged_df.apply(compute_final_consistency, axis=1)

# Sort by Consistency
top_100 = merged_df.sort_values(by='Consistency_Score', ascending=False)[
    ['Player Name', 'Bat_Consistency', 'Bowl_Consistency', 'Consistency_Score']
].head(100)


top_100.to_csv('top_100_consistent_players.csv', index=False)

                  Player  Bat_Consistency  Bowl_Consistency  Consistency_Score
125       Tristan Stubbs        70.597500             100.0          85.298750
122            Tim David        63.783739             100.0          81.891870
113         Shreyas Iyer        60.897436             100.0          80.448718
40             K L Rahul        60.744915             100.0          80.372457
72       Nicholas Pooran        58.894231             100.0          79.447115
12       Ashutosh Sharma        54.803729             100.0          77.401864
132        Vijay Shankar        50.316784             100.0          75.158392
57        Mitchell Marsh        45.183323             100.0          72.591661
71         Nehal Wadhera        43.878974             100.0          71.939487
93           Rinku Singh        43.251346             100.0          71.625673
81         Priyansh Arya        43.212842             100.0          71.606421
101        Sai Sudharsan        42.290833           

,POS_bat,Player,Runs_bat,Mat_bat,Inns_bat,NO,HS,Avg_bat,BF,SR_bat,...,4w,5w,Avg_bat_norm,SR_bat_norm,Bat_Consistency,Avg_bowl_norm,Econ_norm,SR_bowl_norm,Bowl_Consistency,Consistency_Score
0,56.0,Abdul Samad,59.0,4.0,4.0,1.0,27,19.67,26.0,226.92,...,0.0,0.0,16.811966,56.7300,36.770983,100.000000,100.000000,100.000000,100.000000,68.385491
1,99.0,Abhinav Manohar,6.0,3.0,3.0,0.0,4,2.00,13.0,46.15,...,0.0,0.0,1.709402,11.5375,6.623451,100.000000,100.000000,100.000000,100.000000,53.311725
2,61.0,Abhishek Sharma,51.0,5.0,5.0,0.0,24,10.20,40.0,127.50,...,0.0,0.0,8.717949,31.8750,20.296474,100.000000,100.000000,100.000000,100.000000,60.148237
3,54.0,Abishek Porel,74.0,4.0,4.0,1.0,34*,24.67,47.0,157.44,...,0.0,0.0,21.085470,39.3600,30.222735,100.000000,100.000000,100.000000,100.000000,65.111368
4,0.0,Adam Zampa,0.0,0.0,0.0,0.0,0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0000,0.000000,69.078947,16.666667,75.757576,53.834397,26.917198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.0,Yash Dayal,0.0,0.0,0.0,0.0,0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0000,0.000000,83.118421,35.177305,83.000000,67.098575,33.549288
140,0.0,Yash Thakur,0.0,0.0,0.0,0.0,0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0000,0.000000,74.342105,30.851064,75.757576,60.316915,30.158457
141,35.0,Yashasvi Jaiswal,107.0,5.0,5.0,0.0,67,21.40,84.0,127.38,...,0.0,0.0,18.290598,31.8450,25.067799,100.000000,100.000000,100.000000,100.000000,62.533900
142,0.0,Yuzvendra Chahal,0.0,0.0,0.0,0.0,0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0000,0.000000,26.973684,28.439716,33.333333,29.582245,14.791122


In [16]:
import pandas as pd

# Merge on exact player names
merged_df = pd.read_csv('/content/recent_data_type.csv')

# Replace '-' and NaN values with 0 for consistency calculations
merged_df.replace('-', 0, inplace=True)
merged_df.fillna(0, inplace=True)

# Ensure numeric columns
for col in ['Avg_bat', 'SR_bat', 'Avg_bowl', 'Econ_bowl', 'SR_bowl']:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)

# Batting Consistency
merged_df['Bat_Consistency'] = (merged_df['Avg_bat'] + merged_df['SR_bat']) / 2

# Bowling Consistency: 150 - avg of [Avg_bowl, Econ_bowl, SR_bowl], but only if any of them > 0
def compute_bowl_consistency(row):
    if row['Avg_bowl'] == 0 and row['Econ_bowl'] == 0 and row['SR_bowl'] == 0:
        return 0  # No bowling stats, so return 0 for consistency
    return 150 - ((row['Avg_bowl'] + row['Econ_bowl'] + row['SR_bowl']) / 3)

merged_df['Bowl_Consistency'] = merged_df.apply(compute_bowl_consistency, axis=1)

# Apply restrictions based on player type
def apply_role_restrictions(row):
    # If Player Type is 'Batsman' or 'Wicket Keeper', consider only Bat_Consistency
    if row['Player Type'] in ['BAT', 'WK']:
        row['Bowl_Consistency'] = 0  # Set Bowl_Consistency to 0 for Batsmen and Wicket Keepers
    # If Player Type is 'Bowler', consider only Bowl_Consistency
    elif row['Player Type'] == 'BOWL':
        row['Bat_Consistency'] = 0  # Set Bat_Consistency to 0 for Bowlers
    # If Player Type is 'All-rounder', both Bat_Consistency and Bowl_Consistency are considered
    return row

merged_df = merged_df.apply(apply_role_restrictions, axis=1)

# Final Consistency Score: Combine Batting and Bowling Consistency
def compute_final_consistency(row):
    if row['Bat_Consistency'] > 0 and row['Bowl_Consistency'] > 0:
        return (row['Bat_Consistency'] + row['Bowl_Consistency']) / 2
    elif row['Bat_Consistency'] > 0:
        return row['Bat_Consistency']
    elif row['Bowl_Consistency'] > 0:
        return row['Bowl_Consistency']
    return 0

merged_df['Consistency_Score'] = merged_df.apply(compute_final_consistency, axis=1)

# Sort by Consistency
top_100 = merged_df.sort_values(by='Consistency_Score', ascending=False)[
    ['Player Name', 'Player Type', 'Bat_Consistency', 'Bowl_Consistency', 'Consistency_Score']
].head(100)

# Save the top 100 consistent players
top_100.to_csv('Recent_player_stat.csv', index=False)

# Optionally print the top 100 consistent players
top_100


,Player Name,Player Type,Bat_Consistency,Bowl_Consistency,Consistency_Score
72,Nicholas Pooran,WK,148.500,0.000000,148.500000
122,Tim David,BAT,143.870,0.000000,143.870000
112,Shreyas Iyer,BAT,142.000,0.000000,142.000000
125,Tristan Stubbs,BAT,140.890,0.000000,140.890000
45,Kuldeep Yadav,BOWL,0.000,140.440000,140.440000
...,...,...,...,...,...
114,Shubman Gill,BAT,91.765,0.000000,91.765000
44,Krunal Pandya,ALL,48.065,135.033333,91.549167
26,Faf du Plessis,BAT,91.380,0.000000,91.380000
130,Venkatesh Iyer,BAT,91.275,0.000000,91.275000


In [14]:
import pandas as pd

# Load the datasets
d1 = pd.read_csv('/content/SquadPlayerNames_IndianT20League - SquadData_AllTeams.csv')  # This dataset contains 'Player Type'
d2 = pd.read_csv('/content/recent_data.csv')  # This dataset will receive the 'Player Type'

# Strip any leading/trailing whitespace from player names to avoid mismatches
d1['Player Name'] = d1['Player Name'].str.strip()
d2['Player Name'] = d2['Player Name'].str.strip()

# Merge only the 'Player Type' column from d1 into d2 based on 'Player Name'
merged_df = pd.merge(d2, d1[['Player Name', 'Player Type']], on='Player Name', how='left')

# Save the merged dataset to a new CSV file
merged_df.to_csv('recent_data_type.csv', index=False)

# Optionally print the merged dataframe to check
print(merged_df.head())


   POS_bowl      Player Name  Wkts_bowl  Mat_bowl  Inns_bowl  Ov_bowl  \
0       0.0      Abdul Samad        0.0       0.0        0.0      0.0   
1       0.0  Abhinav Manohar        0.0       0.0        0.0      0.0   
2       0.0  Abhishek Sharma        0.0       0.0        0.0      0.0   
3       0.0    Abishek Porel        0.0       0.0        0.0      0.0   
4      53.0       Adam Zampa        2.0       2.0        2.0      8.0   

   Runs_bowl BBI_bowl  Avg_bowl  Econ_bowl  ...  NO_bat  HS_bat  Avg_bat  \
0        0.0        0       0.0       0.00  ...     1.0      27    19.67   
1        0.0        0       0.0       0.00  ...     0.0       4        2   
2        0.0        0       0.0       0.00  ...     0.0      24     10.2   
3        0.0        0       0.0       0.00  ...     1.0     34*    24.67   
4       94.0     46/1      47.0      11.75  ...     0.0       0        0   

   BF_bat  SR_bat  100_bat  50_bat  4s_bat 6s_bat Player Type  
0    26.0  226.92      0.0     0.0     6

In [18]:
import pandas as pd
import numpy as np

# ✅ Load the dataset
df = pd.read_csv("/content/Final_cricket_data.csv")

# ✅ Convert required columns to numeric
num_columns = [
    "Runs_Scored", "Batting_Strike_Rate", "Balls_Faced", "Matches_Batted",
    "Centuries", "Half_Centuries", "Fours", "Sixes", "Recent_Batting_Avg",
    "Matches_Bowled", "Wickets_Taken", "Bowling_Average", "Economy_Rate",
    "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls", "Recent_Bowling_Avg",
    "Catches_Taken", "Stumpings", "Credits"
]
df[num_columns] = df[num_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Fill NaNs with 0
df.fillna(0, inplace=True)

# ✅ Fantasy points calculation
df["Raw_Fantasy_Points"] = (
    df["Runs_Scored"]
    + np.where(df["Runs_Scored"] >= 100, 16, 0)
    + np.where((df["Runs_Scored"] >= 50) & (df["Runs_Scored"] < 100), 8, 0)
    + np.where(df["Batting_Strike_Rate"] >= 140, 6,
               np.where(df["Batting_Strike_Rate"] >= 120, 4,
                        np.where(df["Batting_Strike_Rate"] >= 100, 2, 0)))
    - np.where((df["Runs_Scored"] == 0) & (df["Balls_Faced"] > 0), 2, 0)
    + df["Wickets_Taken"] * 25
    + np.where(df["Four_Wicket_Hauls"] > 0, 8, 0)
    + np.where(df["Five_Wicket_Hauls"] > 0, 16, 0)
    + np.where(df["Economy_Rate"] < 5, 6,
               np.where(df["Economy_Rate"] < 6, 4,
                        np.where(df["Economy_Rate"] < 7, 2,
                                 np.where(df["Economy_Rate"] > 9, -2, 0))))
    + df["Catches_Taken"] * 8
    + df["Stumpings"] * 12
)

# ✅ Normalize by matches played (batting or bowling)
df["Matches_Played"] = df[["Matches_Batted", "Matches_Bowled"]].max(axis=1)
df["Matches_Played"] = df["Matches_Played"].replace(1, 1)

# ✅ Save the dataset
df.to_csv("fantasy_ready_dataset.csv", index=False)
print("✅ Normalized fantasy points saved in 'fantasy_ready_dataset.csv'!")


✅ Normalized fantasy points saved in 'fantasy_ready_dataset.csv'!
